# Purpose

Evaluate ability of max-divergent to recover known divergent lineages. We generate 4 synthetic lineages (referred to as pools in the code) and check whether the result of `max_divergent` contains one representative of each.

# Design

We define 4 distinct sequence compositions. We then defined two distinct "pool" compositions in which each sequence family was present at equal frequency (termed "balanced") or one sequence was very rare (termed "imbalanced"). In each scenario, we simulated a total of 50 sequences. This design is intended to assess whether rare sequences can be recovered.

We expect `divergent max` to identify a set of precisely 4 sequences with one representative of each pool. We examined the influence of statistic choice.

# Result summary

Aside from using the `total_jsd` statistic, perfect recall (one representative from each of 4 distinct pools) was observed for all cases.

In [ ]:
import project_path
from cogent3 import load_table

write_pdf = project_path.pdf_writer()

table = load_table(project_path.RESULT_DIR / "synthetic_known_summary.tsv", digits=2)
table

In [ ]:
import plotly.express as px

# Define custom colors for cov and std
cols = {
    "stdev": "blue",
    "cov": "red",
}

# Create a grouped bar chart
fig = px.bar(
    table.to_pandas(),
    x="seq_len",  # Group bars by seq_len
    y="correct%",  # Height of bars
    color="stat",  # Set legend by stat (cov or std)
    facet_row="pool",  # Separate plots by pool (balanced, imbalanced)
    error_y="stdv(correct)",  # Add error bars from the stdv(correct) column
    category_orders={"seq_len": [200, 1000]},  # Order bars by seq_len
    color_discrete_map=cols,  # Set custom colors for cov and std
    barmode="group",  # Place bars beside each other
    opacity=0.6,
    labels={"correct%": "Correct %", "seq_len": "Sequence Length", "stat": "Statistic"},
)

# Adjust subplot titles to appear on the right border
fig.for_each_annotation(lambda a: a.update(textangle=0, x=1.02, xanchor="left"))

# Update layout for better visualization
tickfont = dict(size=20)
titlefont = dict(size=26)
legend = dict(title="", font=dict(size=28), tracegroupgap=10)

fig.update_layout(
    legend=legend,
    xaxis_title="Sequence Length",
    yaxis_title="Correct %",
    width=1200,
    height=800,
)

fig.update_yaxes(tickfont=tickfont, row=1, col=1, title_font=titlefont)
fig.update_yaxes(tickfont=tickfont, row=2, col=1, title_font=titlefont)
fig.update_xaxes(title_font=titlefont, tickfont=tickfont)

d_jsd = "δ<sub>JSD</sub>"
stat_names = dict(stdev=f"std({d_jsd})", cov=f"cov({d_jsd})", JSD=r"JSD(<b>F</b>)")

for name in ["stdev", "cov"]:
    fig.update_traces(selector={"name": name}, name=stat_names[name])

for ann in fig.layout.annotations:
    ann["textangle"] = 90
    ann["font"].update(tickfont)
    ann["x"] = 0.99

# Show the plot
# fig.show(renderer="notebook")

outpath = project_path.FIG_DIR / "synthetic_known_bar.pdf"
write_pdf(fig, outpath)